# term1_sprint8 アンサンブル学習

## .アンサンブル学習

3種類のアンサンブル学習をスクラッチ実装していきます。そして、それぞれの効果を小さめのデータセットで確認します。


ブレンディング
バギング
スタッキング

小さなデータセットの用意
以前も利用した回帰のデータセットを用意します。


House Prices: Advanced Regression Techniques  
https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data



この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。


train.csvを学習用（train）8割、検証用（val）2割に分割してください。


scikit-learn
単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。


sklearn.linear_model.LinearRegression — scikit-learn 0.21.3 documentation  
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html  


sklearn.svm.SVR — scikit-learn 0.21.3 documentation  
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html  


sklearn.tree.DecisionTreeRegressor — scikit-learn 0.21.3 documentation  
https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

### 0.0.0（ベースデータ準備)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter('ignore')

from sklearn .linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv("sample_dataset/house-prices-advanced-regression-techniques/train.csv")

X = data[["GrLivArea","YearBuilt"]]
Y = data[["SalePrice"]]

In [3]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   GrLivArea  1460 non-null   int64
 1   YearBuilt  1460 non-null   int64
dtypes: int64(2)
memory usage: 22.9 KB


## 【問題1】ブレンディングのスクラッチ実装
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。


ブレンディングとは
ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。


手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
入力データの前処理の仕方（例：標準化、対数変換、PCAなど）

重要なのはそれぞれのモデルが大きく異なることです。


回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。


《補足》


分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。


sklearn.ensemble.VotingClassifier — scikit-learn 0.21.3 documentation

### 1.1.1（予備知識）単体scoreの検証

In [4]:
def score_mean(clf, X, Y, X_val=None, Y_val=None, num=100):
    clf_score_list = []
    clf_error_list = []
    
    for i in range(num):
        x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
        
        clf = clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        
        clf_score_list.append(clf.score(x_test, y_test))
        clf_error_list.append(mean_squared_error(y_test, y_pred))
    
    mean = np.mean(clf_score_list)
    error = np.mean(clf_error_list)
    
    return mean, error

In [5]:
# 線形回帰
line = LinearRegression()
line_data = score_mean(line, X, Y)
print("line mean:", line_data[0], "error", line_data[1])

line mean: 0.6400433248652061 error 2234579950.0787716


In [6]:
# SVM
SVM = SVR(kernel='linear')
SVM_data = score_mean(SVM, X, Y)
print("SVM mean:", SVM_data[0], "error", SVM_data[1])

SVM mean: 0.6340332587038313 error 2266042039.8533535


In [7]:
# 決定木
tree = DecisionTreeRegressor()
tree_data = score_mean(tree, X, Y)
print("tree mean:", tree_data[0], "error", tree_data[1])

tree mean: 0.5475747468401644 error 2803662846.468687


### 1.2.1（解答)ブレンディング用関数

In [13]:
class blending():
    def __init__(self, clf1, clf2, ratio1, ratio2):
        self.clf1 = clf1
        self.clf2 = clf2
        x = ratio1
        y = ratio2
        self.ratio1 = x/(x+y)
        self.ratio2 = y/(x+y)
    
    def fit(self, X, Y):
        self.clf1.fit(X, Y)
        self.clf2.fit(X, Y)

    def predict(self, X_var):
        pred_1 = (self.clf1.predict(X_var) * self.ratio1).reshape(-1,1)
        pred_2 = (self.clf2.predict(X_var) * self.ratio2).reshape(-1,1)
        return pred_1+pred_2
        
    def score(self, X_var, Y_var):
        score1 = self.clf1.score(X_var, Y_var) * self.ratio1
        score2 = self.clf2.score(X_var, Y_var) * self.ratio2
        return score1+score2

### 1.3.1（解答）blend_scoreの平均算出用関数

In [14]:
def blend_score_mean(clf1, clf2, ratio1, ratio2, X, Y, num=100):
    
    score_list = []
    error_list = []
    
    for i in range(num):
        x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
        
        model = blending(clf1, clf2, ratio1, ratio2)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        
        score_list.append(model.score(x_test, y_test))
        error_list.append(mean_squared_error(y_test, y_pred))
        
    mean = np.mean(score_list)
    error = np.mean(error_list)
    
    return mean, error

### 1.4.1（解答）blend検証

In [20]:
# 線形回帰 ＋ SVM
ls1 = blend_score_mean(line, SVM, line_data[0], SVM_data[0], X, Y)
print("line_SVM mean:", ls1[0], "error", ls1[1])

line_SVM mean: 0.6419621450836465 error 2229186093.2301583


In [18]:
# SVM + tree
st2 = blend_score_mean(SVM, tree, SVM_data[0], tree_data[0], X, Y)
print("SVM_tree mean:", st2[0], "error", st2[1])

SVM_tree mean: 0.5920884264300557 error 2016288598.308999


In [19]:
# tree + line
tl3 = blend_score_mean(tree, line, tree_data[0], line_data[0], X, Y)
print("tree_line mean:", tl3[0], "error", tl3[1])

tree_line mean: 0.6069852466071737 error 1907580092.3443573


# 4.バギング

## 【問題2】バギングのスクラッチ実装
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。


バギングとは
バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。


sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation  
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html


scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。


推定結果の平均をとる部分はブースティングと同様の実装になります。


### 2.1.1（解答）

In [22]:
class bagging():
    def fit(self, X, Y): 
        x_1a, x_1b, y_1a, y_1b = train_test_split(X, Y, test_size=0.2, shuffle = True)
        x_2a, x_2b, y_2a, y_2b = train_test_split(X, Y, test_size=0.2, shuffle = True)
        x_3a, x_3b, y_3a, y_3b = train_test_split(X, Y, test_size=0.2, shuffle = True)
        x_4a, x_4b, y_4a, y_4b = train_test_split(X, Y, test_size=0.2, shuffle = True)
        
        self.tree1 = DecisionTreeRegressor()
        self.tree2 = DecisionTreeRegressor()
        self.tree3 = DecisionTreeRegressor()
        self.tree4 = DecisionTreeRegressor()
        
        self.tree1 = self.tree1.fit(x_1a, y_1a)
        self.tree2 = self.tree2.fit(x_2a, y_2a)
        self.tree3 = self.tree3.fit(x_3a, y_3a)
        self.tree4 = self.tree4.fit(x_4a, y_4a)
        
    def predict(self, X):
        pred_1 = self.tree1.predict(X).reshape(-1,1)
        pred_2 = self.tree2.predict(X).reshape(-1,1)
        pred_3 = self.tree3.predict(X).reshape(-1,1)
        pred_4 = self.tree4.predict(X).reshape(-1,1)

        return (pred_1+pred_2+pred_3+pred_4)/4
    
    def score(self, X, Y):
        score_1 = self.tree1.score(X, Y)
        score_2 = self.tree2.score(X, Y)
        score_3 = self.tree3.score(X, Y)
        score_4 = self.tree4.score(X, Y)
        score_list = [score_1, score_2, score_3, score_4]
        
        return np.mean(score_list)

In [23]:
def bagging_score_mean(X, Y, num=100):
    
    score_list = []
    error_list = []
    
    for i in range(num):
        x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
        
        bagg = bagging()
        bagg.fit(x_train, y_train)
        y_pred = bagg.predict(x_test)
        
        score_list.append(bagg.score(x_test, y_test))
        error_list.append(mean_squared_error(y_test, y_pred))
        
    mean = np.mean(score_list)
    error = np.mean(error_list)
    
    return mean, error

In [24]:
# 4tree_bagging
bagg = bagging_score_mean(X, Y)
print("bagging mean:", bagg[0], "error", bagg[1])

bagging mean: 0.5254202346183174 error 2291607696.6303396


# 5.スタッキング

## 【問題3】スタッキングのスクラッチ実装
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。


スタッキングとは
スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは 
K
0
=
3
,
M
0
=
2
 程度にします。


《学習時》


（ステージ 
0
 ）


学習データを 
K
0
 個に分割する。
分割した内の 
(
K
0
−
1
)
 個をまとめて学習用データ、残り 
1
 個を推定用データとする組み合わせが 
K
0
 個作れる。
あるモデルのインスタンスを 
K
0
 個用意し、異なる学習用データを使い学習する。
それぞれの学習済みモデルに対して、使っていない残り 
1
 個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）
さらに、異なるモデルのインスタンスも 
K
0
 個用意し、同様のことを行う。モデルが 
M
0
 個あれば、 
M
0
 個のブレンドデータが得られる。

（ステージ 
n
 ）


ステージ 
n
−
1
 のブレンドデータを
M
n
−
1
 次元の特徴量を持つ学習用データと考え、 
K
n
 個に分割する。以下同様である。

（ステージ 
N
 ）＊最後のステージ


ステージ 
N
−
1
 の 
M
N
−
1
 個のブレンドデータを
M
N
−
1
 次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

《推定時》


（ステージ 
0
 ）


テストデータを 
K
0
×
M
0
 個の学習済みモデルに入力し、
K
0
×
M
0
 個の推定値を得る。これを 
K
0
 の軸で平均値を求め 
M
0
 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ 
n
 ）


ステージ 
n
−
1
 で得たブレンドテストを 
K
n
×
M
n
 個の学習済みモデルに入力し、
K
n
×
M
n
 個の推定値を得る。これを 
K
n
 の軸で平均値を求め 
M
0
 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ 
N
 ）＊最後のステージ


ステージ 
N
−
1
 で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

### 3.1.1（解答） stage0の学習用関数

In [25]:
from sklearn.model_selection import KFold

class staking():
    def __init__(self, model, splits=3):
        # 分類機を設定
        self.model = model
        self.splits = splits
        # K-Folds(クロスバリデーション)の設定
        self.kf = KFold(n_splits=self.splits, random_state=None, shuffle=False)
        
    def fit(self, X, Y):
        self.model.fit(X, Y)
        
    def predict(self, X):
        return self.model.predict(X)
    
    # stage0の学習→分類器毎のoof（out of fold)を返す----------------------
    
    def get_oof(self, x_train, y_train, x_test):
        # oofの受け皿を作成
        oof_train = np.zeros((x_train.shape[0],))
        oof_test = np.zeros((x_test.shape[0],))
        oof_test_skf = np.empty((self.splits, x_test.shape[0]))
        
        # K-Foldsでデータを分ける
        for i, (train_index, test_index) in enumerate(self.kf.split(x_train, y_train)):
            x_tr = x_train[train_index]
            y_tr = y_train[train_index]
            x_te = x_train[test_index]
#             print(i, "x_tr", x_tr, "y_tr", y_tr, "x_te", x_te)
            
            self.model.fit(x_tr, y_tr)
            
            # 学習結果を元にtrainデータ(test)の推定
            oof_train[test_index] = self.model.predict(x_te).ravel()
            # 学習結果を元にtestデータの推定
            oof_test_skf[i, :] = self.model.predict(x_test).ravel()
            
        oof_test[:] = oof_test_skf.mean(axis=0)
        
        return (oof_train.reshape(-1, 1), oof_test.reshape(-1,1))    

### 3.1.2（解答） stage0の実行

In [28]:
# 使用する分類機の呼び出し
dt = staking(model=DecisionTreeRegressor(), splits=4)
sv = staking(model=SVR(kernel='linear'), splits=4)
lr = staking(model=LinearRegression(), splits=4)

In [29]:
# stage0の実行
lr_oof_train, lr_oof_test = lr.get_oof(x_train, y_train, x_test)
dt_oof_train, dt_oof_test = dt.get_oof(x_train, y_train, x_test)
sv_oof_train, sv_oof_test = sv.get_oof(x_train, y_train, x_test)

### 3.2.1（解答） stage1の実行

In [30]:
# stage1用データの用意
x_train_oof = np.concatenate((lr_oof_train, dt_oof_train, sv_oof_train),axis=1)
x_test_oof = np.concatenate((lr_oof_test, dt_oof_test, sv_oof_test),axis=1)
print("x_train_oof",x_train_oof.shape)
print("x_test_oof ",x_test_oof.shape)

x_train_oof (1168, 3)
x_test_oof  (292, 3)


In [31]:
# stage1学習の実行
dt2 = DecisionTreeRegressor()
dt2.fit(x_train_oof, y_train)
dt2_pred = dt2.predict(x_test_oof)
dt2_score = dt2.score(x_test_oof, y_test)
error = mean_squared_error(y_test, dt2_pred)
print("score",dt2_score, "error", error)

score 0.5103416273740732 error 3164153212.7260275


### 3.3.1（解答）平均値

In [32]:
dt2_score_list = []
dt2_error_list = []

for i in range(20):
    X = np.array(X)
    Y = np.array(Y)
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
    
    dt = staking(model=DecisionTreeRegressor(), splits=4)
    sv = staking(model=SVR(kernel='linear'), splits=4)
    lr = staking(model=LinearRegression(), splits=4)
    
    lr_oof_train, lr_oof_test = lr.get_oof(x_train, y_train, x_test)
    dt_oof_train, dt_oof_test = dt.get_oof(x_train, y_train, x_test)
    sv_oof_train, sv_oof_test = sv.get_oof(x_train, y_train, x_test)
    
    x_train_oof = np.concatenate((lr_oof_train, dt_oof_train, sv_oof_train),axis=1)
    x_test_oof = np.concatenate((lr_oof_test, dt_oof_test, sv_oof_test),axis=1)
    
    dt2 = DecisionTreeRegressor()
    dt2.fit(x_train_oof, y_train)
    dt2_pred = dt2.predict(x_test_oof)
    dt2_score_list.append(dt2.score(x_test_oof, y_test))
    dt2_error_list.append(mean_squared_error(y_test, dt2_pred))

dt2_mean = np.mean(dt2_score_list)
dt2_error = np.mean(dt2_error_list)
dt2 = [dt2_mean, dt2_error]

print("dt2 score",dt2[0], "error", dt2[1])

dt2 score 0.4579333452079763 error 3535936527.7864633


In [33]:
SVM2_score_list = []
SVM2_error_list = []

for i in range(1):
    X = np.array(X)
    Y = np.array(Y)
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
    
    dt = staking(model=DecisionTreeRegressor(), splits=4)
    sv = staking(model=SVR(kernel='linear'), splits=4)
    lr = staking(model=LinearRegression(), splits=4)
    
    lr_oof_train, lr_oof_test = lr.get_oof(x_train, y_train, x_test)
    dt_oof_train, dt_oof_test = dt.get_oof(x_train, y_train, x_test)
    sv_oof_train, sv_oof_test = sv.get_oof(x_train, y_train, x_test)
    
    x_train_oof = np.concatenate((lr_oof_train, dt_oof_train, sv_oof_train),axis=1)
    x_test_oof = np.concatenate((lr_oof_test, dt_oof_test, sv_oof_test),axis=1)
    
    SVM2 = SVR(kernel='linear')
    SVM2.fit(x_train_oof, y_train)
    SVM2_pred = SVM2.predict(x_test_oof)
    SVM2_score_list.append(SVM2.score(x_test_oof, y_test))
    SVM2_error_list.append(mean_squared_error(y_test, SVM2_pred))

SVM2_mean = np.mean(SVM2_score_list)
SVM2_error = np.mean(SVM2_error_list)
SVM2 = [SVM2_mean, SVM2_error]

print("SVM2 score",SVM2[0], "error", SVM2[1])

SVM2 score 0.6256222525056039 error 2435809290.5139647


### 3.4.1（予備知識）結果確認

In [34]:
data = np.array([line_data, SVM_data, tree_data, ls1, st2, tl3, bagg, dt2, SVM2])

df = pd.DataFrame(data)
df.columns = ["score", "MSE(平均乗誤差)"]
df.index = ["線形回帰", "SVM", "決定木", "線形回帰+SVM", "SVM＋決定木", "決定木＋線形回帰", "Bagging","Staking_決定木","Staking_SVM"]
df.sort_values("MSE(平均乗誤差)")

,score,MSE(平均乗誤差)
決定木＋線形回帰,0.606985,1.907580e+09
SVM＋決定木,0.592088,2.016289e+09
線形回帰+SVM,0.641962,2.229186e+09
線形回帰,0.640043,2.234580e+09
SVM,0.634033,2.266042e+09
Bagging,0.525420,2.291608e+09
Staking_SVM,0.625622,2.435809e+09
決定木,0.547575,2.803663e+09
Staking_決定木,0.457933,3.535937e+09


### 検証保管